# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,64.076,72,81,4.79,AU,1703633236
1,1,ushuaia,-54.8000,-68.3000,51.458,46,0,14.92,AR,1703633286
2,2,veinticinco de mayo,-35.4323,-60.1727,70.952,55,0,2.81,AR,1703633289
3,3,straseni,47.1414,28.6103,44.420,89,23,4.26,MD,1703633291
4,4,katsuura,35.1333,140.3000,48.686,65,79,4.31,JP,1703633291


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points("Lng", "Lat", geo = True,
                                          tiles = "OSM", size="Humidity", color="City", scale=0.7)

# Display the map
humidity_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
##Max Temp lower than 100 but higher than 65
##Wind speed less than 5 mph
##Zero Cloudiness

# YOUR CODE HERE
filtered_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 100) &
    (city_data_df["Max Temp"] > 65) &
    (city_data_df["Wind Speed"] < 10) &
    (city_data_df["Cloudiness"] == 0)
]


# Drop any rows with null values
# YOUR CODE HERE
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,veinticinco de mayo,-35.4323,-60.1727,70.952,55,0,2.81,AR,1703633289
47,47,saint-pierre,-21.3393,55.4781,71.276,64,0,4.12,RE,1703633331
50,50,mount isa,-20.7333,139.5000,88.016,10,0,4.88,AU,1703633334
75,75,hawaiian paradise park,19.5933,-154.9731,80.114,79,0,4.63,US,1703633354
132,132,chonchi,-42.6128,-73.8092,65.696,53,0,2.59,CL,1703633386


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,veinticinco de mayo,AR,-35.4323,-60.1727,55,
47,saint-pierre,RE,-21.3393,55.4781,64,
50,mount isa,AU,-20.7333,139.5000,10,
75,hawaiian paradise park,US,19.5933,-154.9731,79,
132,chonchi,CL,-42.6128,-73.8092,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "type": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row["Lat"], row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["filter"] = f"proximity:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    #print(name_address)
    
     # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
veinticinco de mayo - nearest hotel: Jorly
saint-pierre - nearest hotel: Tropic Hotel
mount isa - nearest hotel: Ibis Styles
hawaiian paradise park - nearest hotel: Lava Tree Tropic Inn
chonchi - nearest hotel: Hotel Huildín
port macquarie - nearest hotel: The Observatory
brisas de zicatela - nearest hotel: Casa de Olas
hermosillo - nearest hotel: Lucerna
barra dos coqueiros - nearest hotel: Grande Hotel
boke - nearest hotel: Hôtel Rio Nunez
buenavista del norte - nearest hotel: Melia Hacienda del Conde
la reforma - nearest hotel: Hotel Flores
nova sintra - nearest hotel: Residência Ka Dencho
az zawiyah - nearest hotel: علاء مرداوي
hilo - nearest hotel: Dolphin Bay Hotel
san isidro - nearest hotel: Soul Mate
mar del plata - nearest hotel: Gran hotel Mónaco
upington - nearest hotel: Die Eiland Holiday Ressort
bubaque - nearest hotel: Chez Julio
el-tor - nearest hotel: No hotel found
umluj - nearest hotel: Nujuma, a Ritz-Carlton Reserve
cabo san lucas - nearest hote

,City,Country,Lat,Lng,Humidity,Hotel Name
2,veinticinco de mayo,AR,-35.4323,-60.1727,55,Jorly
47,saint-pierre,RE,-21.3393,55.4781,64,Tropic Hotel
50,mount isa,AU,-20.7333,139.5000,10,Ibis Styles
75,hawaiian paradise park,US,19.5933,-154.9731,79,Lava Tree Tropic Inn
132,chonchi,CL,-42.6128,-73.8092,53,Hotel Huildín
145,port macquarie,AU,-31.4333,152.9167,36,The Observatory
146,brisas de zicatela,MX,15.8369,-97.0419,58,Casa de Olas
188,hermosillo,MX,29.0667,-110.9667,22,Lucerna
209,barra dos coqueiros,BR,-10.9089,-37.0386,83,Grande Hotel
286,boke,GN,10.9409,-14.2967,79,Hôtel Rio Nunez


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
%%capture --no-display

# Configure the map plot
humidity_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    hover_cols=["City", "Humidity", "Hotel Name", "Country"],
    scale=0.8,
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)